<a href="https://colab.research.google.com/github/kktsuji/train-lora-cell-rois/blob/main/generate_cell_rois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from diffusers import StableDiffusionPipeline
import torch
from peft import PeftModel

In [3]:
# Variable Settings

# RESOLUTION = "40" # original size
# RESOLUTION = "256"
RESOLUTION = "512" # recommended for sd-1.x

NUM_TRAIN_EPOCHS = "10"

lora_dir_name = f"cell-rois_lora_resolution-{RESOLUTION}_epochs-{NUM_TRAIN_EPOCHS}"

In [4]:
import os

lora_path = f"/content/drive/MyDrive/Research/LoRAs/{lora_dir_name}/unet/"
print(lora_path)

model_id = "CompVis/stable-diffusion-v1-4"
model_dir_path = "/content/drive/MyDrive/Research/Models/" + model_id
print(model_dir_path)

out_dir = f"/content/drive/MyDrive/Research/Results/{lora_dir_name}/"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
print(out_dir)

if (os.path.exists(lora_path) or
  os.path.exists(model_dir_path) or
  os.path.exists(out_dir)):
  print("All directories exist")
else:
  from google.colab import runtime
  runtime.unassign()
  raise Exception("Directory not found")

/content/drive/MyDrive/Research/LoRAs/cell-rois_lora_resolution-512_epochs-10/unet/
/content/drive/MyDrive/Research/Models/CompVis/stable-diffusion-v1-4
/content/drive/MyDrive/Research/Results/cell-rois_lora_resolution-512_epochs-10/
All directories exists


In [5]:
seed_num = 10
prompt = "black background"
lora_id = "rsd"
# prompt_lora = lora_id + " " + prompt
prompt_lora = lora_id

In [6]:
pipe = StableDiffusionPipeline.from_pretrained(model_dir_path, torch_dtype=torch.float16)
pipe.to("cuda")

original_unet = pipe.unet

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
def generate_and_save(pipe, prompt, seed, out_dir, suffix):
  image = pipe(
      prompt,
      generator=torch.Generator("cuda").manual_seed(seed)
      ).images[0]
  image.save(out_dir + f"seed{seed}_" + prompt.replace(" ", "_") + f"_{suffix}.png")

In [8]:
# for seed in range(seed_num):
#   print(f"seed: {seed} / {seed_num - 1}")

#   generate_and_save(pipe, prompt, seed, out_dir, "ori")
#   generate_and_save(pipe, prompt_lora, seed, out_dir, "ori")

In [9]:
lora_unet = PeftModel.from_pretrained(original_unet, lora_path, adapter_name=lora_id)
pipe.unet = lora_unet
pipe.to("cuda")

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.34.0",
  "_name_or_path": "/content/drive/MyDrive/Research/Models/CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [10]:
for seed in range(seed_num):
  print(f"seed: {seed} / {seed_num - 1}")

  # generate_and_save(pipe, prompt, seed, out_dir, "lora")
  generate_and_save(pipe, prompt_lora, seed, out_dir, "lora")

seed: 0 / 9


  0%|          | 0/50 [00:00<?, ?it/s]

seed: 1 / 9


  0%|          | 0/50 [00:00<?, ?it/s]

seed: 2 / 9


...

  0%|          | 0/50 [00:00<?, ?it/s]

seed: 8 / 9


  0%|          | 0/50 [00:00<?, ?it/s]

seed: 9 / 9


  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
from google.colab import runtime
runtime.unassign()